In [2]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [3]:
# github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
# url = f"{github_url}?raw=1"
df = pd.read_csv("data/results-gpt4o-mini.csv")
df = df.iloc[:300]
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [10]:
df.shape

(300, 5)

In [9]:
len(df["answer_orig"].unique())

60

## Q1

In [3]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

In [7]:
answer_llm = df.iloc[0].answer_llm
print(embedding_model.encode(answer_llm)[0])

-0.42244658


## Q2

In [7]:
def calc_dot_product(row: pd.Series, model: SentenceTransformer):
    v_llm = model.encode(row["answer_llm"])
    v_orig = model.encode(row["answer_orig"])

    return v_llm.dot(v_orig)

In [10]:
df["dot_product"] = df[["answer_llm", "answer_orig"]].apply(
    calc_dot_product, axis=1, args=(embedding_model,)
)

In [12]:
df["dot_product"].quantile(0.75)

np.float64(31.674304962158203)

## Q3

In [14]:
def calc_dot_product_norm(row: pd.Series, model: SentenceTransformer):
    v_llm = model.encode(row["answer_llm"])
    v_orig = model.encode(row["answer_orig"])
    
    llm_norm = np.sqrt((v_llm * v_llm).sum())
    v_norm_llm = v_llm / llm_norm
    
    orig_norm = np.sqrt((v_orig * v_orig).sum())
    v_norm_orig = v_orig / orig_norm

    return v_norm_llm.dot(v_norm_orig)

In [15]:
df["dot_product_norm"] = df[["answer_llm", "answer_orig"]].apply(
    calc_dot_product_norm, axis=1, args=(embedding_model,)
)

In [16]:
df["dot_product_norm"].quantile(0.75)

np.float64(0.8362347632646561)

## Q4

In [4]:
df.iloc[[10]]

,answer_llm,answer_orig,document,question,course
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp


In [5]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[[10]]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Q5

In [8]:
f_scores = [score["f"] for score in scores.values()]
np.mean(f_scores)

np.float64(0.35490034990035496)

## Q6

In [28]:
def calc_rogue_score(row: pd.Series) -> float:
    rouge_scorer = Rouge()

    scores = rouge_scorer.get_scores(row["answer_llm"], row["answer_orig"])[0]

    return scores["rouge-2"]["f"]
    # return np.mean(list(scores["rouge-2"].values()))

In [29]:
df["f_rogue_score"] = df[["answer_llm", "answer_orig"]].apply(
    calc_rogue_score,
    axis=1,
)
df.head()

,answer_llm,answer_orig,document,question,course,rogue_score
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.028169
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.055556
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.177778
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0.047059
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.033898


In [30]:
df["f_rogue_score"].mean()

np.float64(0.20696501983423318)